In [ ]:
import numpy as np
import tensorflow as tf
from pinn import get_network
from pinn.utils import pi_named, atomic_dress, connect_dist_grad
from pinn.models.base import export_model, get_train_op, MetricsCollector
import os, warnings
import tensorflow as tf
from glob import glob
from ase.collections import g2
from pinn.io import load_qm9, sparse_batch
from pinn import get_model, get_calc

Init Plugin
Init Graph Optimizer
Init Kernel


In [ ]:
dataset1 = load_qm9(filelist, splits={'train':8, 'test':2})
for molecule in dataset1['train']:
    print(molecule) #data from qm9 files
    break
for molecule in dataset1['train'].repeat().shuffle(1000).apply(sparse_batch(100)):
    print(molecule) #dataset has been repeated and shuffled
    break
molecule = network.preprocess(molecule)
print(molecule) #now includes one-hot embedding etc

{'elems': <tf.Tensor: shape=(19,), dtype=int32, numpy=
array([6, 6, 6, 7, 7, 6, 6, 8, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int32)>, 'coord': <tf.Tensor: shape=(19, 3), dtype=float32, numpy=
array([[ 0.12453828,  1.3613629 , -0.4989974 ],
       [-0.12656163,  0.10827199,  0.35607484],
       [-1.4915951 , -0.47954565,  0.15802407],
       [-2.393762  , -0.42546415,  1.1326509 ],
       [-3.503685  , -1.0339637 ,  0.6254009 ],
       [-4.7165313 , -1.1230104 ,  1.4049323 ],
       [-3.3004317 , -1.4496067 , -0.6441173 ],
       [-4.3128505 , -2.070327  , -1.2940868 ],
       [-2.0087893 , -1.1181273 , -0.99810946],
       [ 1.1302432 ,  1.7573676 , -0.32552806],
       [ 0.03032281,  1.1396557 , -1.5670043 ],
       [-0.59819144,  2.146699  , -0.25753975],
       [ 0.63609445, -0.6458262 ,  0.12352705],
       [-0.01897491,  0.3539689 ,  1.4169384 ],
       [-5.2713413 , -0.17838433,  1.3889738 ],
       [-5.34652   , -1.9130049 ,  0.9938531 ],
       [-4.453515  , -1.3603593 ,

In [ ]:
filelist = glob('/Users/miguelnavaharris/Project/QM9/*.xyz')
dataset = lambda: load_qm9(filelist, splits={'train':8, 'test':2}) #lambda is being used here to assign execution of load_qm9 to the variable "dataset"
train = lambda: dataset()['train'].repeat().shuffle(1000).apply(sparse_batch(100))
test = lambda: dataset()['test'].repeat().apply(sparse_batch(100))

<ParallelMapDataset shapes: {elems: (None,), coord: (None, 3), e_data: (None,), ind_1: (None, 1)}, types: {elems: tf.int32, coord: tf.float32, e_data: tf.float32, ind_1: tf.int32}>

In [ ]:
for molecule in train():
    print(molecule)
    break


{'elems': <tf.Tensor: shape=(1781,), dtype=int32, numpy=
array([6, 6, 6, 7, 6, 7, 8, 6, 6, 1, 1, 1, 1, 1, 1, 6, 7, 6, 6, 6, 8, 6,
       6, 8, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 8, 8, 6, 6, 8, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 7, 6, 6, 9, 7, 6, 9, 1, 1, 1, 1, 6,
       6, 6, 6, 8, 6, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 8, 6, 6, 8, 6, 6, 6, 6, 1, 1, 1, 1, 6, 6, 6, 8, 6, 6, 6, 8,
       6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 6, 8, 7, 6, 6, 6, 6, 6,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 6, 6, 6, 8, 6, 6, 6, 7, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 6, 6, 8, 6, 6, 8, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1,
       6, 6, 6, 6, 6, 6, 8, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 8, 6, 6, 6, 6, 6, 7, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 6, 6,
       7, 6, 6, 8, 6, 7, 1, 1, 1, 1, 1, 1, 1, 1, 8, 6, 6, 6, 6, 1, 1, 1,
       1, 1, 1, 8, 6, 6, 7, 6, 7, 6, 7, 8, 1, 7, 6, 6, 6, 6, 6, 8, 6, 7,
       1, 1, 1, 1, 1, 1, 1, 1, 7, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 

In [ ]:
params = {'optimizer': {'class_name': 'Adam', 'config': {'learning_rate': {'class_name': 'ExponentialDecay', 'config': {'initial_learning_rate': 0.0003, 'decay_steps': 10000, 'decay_rate': 0.994}}, 'clipnorm': 0.01}}, 'model_dir': '/Users/miguelnavaharris/Project/miguelmodels/deftrainstep', 'network': {'name': 'PiNet', 'params': {'depth': 4, 'rc': 4.0, 'atom_types': [1, 6, 7, 8, 9]}}, 'model': {'name': 'potential_model', 'params': {'learning_rate': 0.001}}}

In [ ]:
network = get_network(params['network'])
print(network)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-04-13 15:41:38.834311: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-13 15:41:38.834416: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
type(network)
isinstance(network, tf.keras.Model)

True

In [ ]:
default_params = {
    ### Scaling and units # The loss function will be MSE((pred - label) * scale)
    # For vector/tensor predictions
    # the error will be pre-component instead of per-atom
    # e_unit is the unit of energy to report w.r.t the input labels
    # no f_unit yet, f_unit is just e_unit/input coordinate unit
    # e.g. if one have input in Hartree, scales it by 100 for training
    #      and output eV when report error
    #      then e_scale should be 100, and e_unit = hartree2evp
    'e_dress': {},  # element-specific energy dress
    'e_scale': 1.0,  # energy scale for prediction
    'e_unit': 1.0,  # output unit of energy during prediction
    # Loss function options
    ## Energy
    'max_energy': False,        # if set to float, omit energies larger than it
    'use_e_per_atom': False,    # use e_per_atom to calculate e_loss
    'log_e_per_atom': False,    # log e_per_atom and its distribution
                                # ^- this is forcely done if use_e_per_atom
    'use_e_weight': False,      # scales the loss according to e_weight
    ## Force
    'use_force': False,         # include force in loss function
    'max_force_comp': False,    # if set to float, omit forces components larger than it
    'no_force_comp': False,     # if set to int, use as maximum number of force components for a update
    'use_f_weight': False,      # scales the loss according to f_weights
    ## Stress
    'use_stress': False,        # include stress in Loss function
    ## L2
    'use_l2': False,
    ## Loss function multipliers
    'e_loss_multiplier': 1.0,
    'f_loss_multiplier': 1.0,
    's_loss_multiplier': 1.0,
    'l2_loss_multiplier': 1.0,
    'separate_errors': False,   # workaround at this point
}

In [ ]:
model_params = default_params.copy()
model_params.update(params['model']['params'])

In [ ]:
print(dataset)
print(type(dataset()))
print('Here is the dataset dictionary:', dataset())
print('The train set is:', dataset()['train'])
pos = 0
for molecule in train():
    pos += 1
    print(molecule)
    if pos == 2:
        break

<function <lambda> at 0x1046dd790>
<class 'dict'>
Here is the dataset dictionary: {'train': <FlatMapDataset shapes: {elems: (None,), coord: (None, 3), e_data: ()}, types: {elems: tf.int32, coord: tf.float32, e_data: tf.float32}>, 'test': <FlatMapDataset shapes: {elems: (None,), coord: (None, 3), e_data: ()}, types: {elems: tf.int32, coord: tf.float32, e_data: tf.float32}>}
The train set is: <FlatMapDataset shapes: {elems: (None,), coord: (None, 3), e_data: ()}, types: {elems: tf.int32, coord: tf.float32, e_data: tf.float32}>
{'elems': <tf.Tensor: shape=(1802,), dtype=int32, numpy=array([6, 6, 8, ..., 1, 1, 1], dtype=int32)>, 'coord': <tf.Tensor: shape=(1802, 3), dtype=float32, numpy=
array([[ 0.24619766,  1.4595945 ,  0.00896299],
       [-0.10159193, -0.03528417,  0.02445348],
       [-1.5189813 , -0.21226592,  0.0091053 ],
       ...,
       [-2.9021015 , -2.8350043 ,  0.08488169],
       [-1.6899433 , -5.106423  , -3.2747953 ],
       [-2.1730773 , -0.27505395,  0.48399863]], dtype=

In [ ]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
print(molecule['ind_1'].numpy())

[[ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 1]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 2]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 7]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 8]
 [ 9]
 [ 9]
 [ 9]
 [ 9]
 [ 9]
 [ 9

In [ ]:
np.unique(molecule['ind_1'].numpy(), return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
       dtype=int32),
 array([17, 16, 17, 19, 23, 14, 16, 25, 14, 16, 17, 15, 11, 18, 20, 22, 19,
        16, 21, 22, 13, 21, 17, 17, 17, 21, 17, 21, 18, 23, 21, 27, 21, 18,
        21, 19, 19, 23, 20, 14, 12, 15, 14, 13, 18, 14, 20, 17, 16, 19, 17,
        21, 20, 16, 18, 19, 19, 16, 16, 17, 19, 19, 19, 19, 15, 18, 19, 19,
        17, 18, 20, 17, 11, 20, 25, 17, 15, 16, 18, 18, 20, 17, 20, 18, 16,
        20, 23, 25, 12, 15, 10, 20, 20, 18, 17, 20, 19, 16, 21, 19]))

In [ ]:
print(molecule)

{'elems': <tf.Tensor: shape=(1709,), dtype=int32, numpy=array([6, 8, 6, ..., 1, 1, 1], dtype=int32)>, 'coord': <tf.Tensor: shape=(1709, 3), dtype=float32, numpy=
array([[ 0.17546003,  1.3340498 ,  0.16630386],
       [-0.00694871, -0.05237263, -0.01236154],
       [ 0.5560965 , -0.5448151 , -1.2099843 ],
       ...,
       [-2.146351  , -6.312405  , -0.4266218 ],
       [-1.1491603 , -6.3121495 ,  1.0367721 ],
       [-2.789544  , -5.647571  ,  1.0922194 ]], dtype=float32)>, 'e_data': <tf.Tensor: shape=(100,), dtype=float32, numpy=
array([-385.77267, -375.10504, -385.8675 , -424.21487, -343.29654,
       -387.10065, -455.1823 , -399.76016, -436.7018 , -417.97174,
       -362.6575 , -358.90292, -440.29428, -458.97852, -363.8613 ,
       -346.53357, -439.03104, -423.0245 , -387.09177, -271.6112 ,
       -441.52182, -455.10867, -426.89865, -424.226  , -383.269  ,
       -440.23697, -401.82056, -342.81412, -417.9624 , -398.1528 ,
       -420.82043, -420.39893, -453.98367, -310.6322 , -460.

In [ ]:
features = network.preprocess(molecule)


In [ ]:
molecule['e_data']

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([-423.0028 , -422.99048, -564.3866 , -395.9951 , -437.90973,
       -455.13776, -424.2457 , -348.68332, -492.00888, -423.00595,
       -439.0764 , -439.05432, -387.0078 , -414.20352, -439.13803,
       -400.6426 , -313.10837, -417.97174, -424.183  , -404.38055,
       -448.92444, -380.89655, -401.90796, -421.8509 , -387.04306,
       -477.95935, -401.85144, -383.42328, -396.8957 , -385.88538,
       -460.1732 , -384.97427, -400.619  , -343.94846, -422.97296,
       -422.98624, -439.16763, -383.64566, -439.08743, -364.67136,
       -380.82315, -399.3608 , -455.19006, -436.70175, -415.41687,
       -470.0391 , -421.75266, -403.1305 , -404.28012, -423.04935,
       -424.17764, -365.96396, -389.52267, -422.95248, -399.83838,
       -384.96286, -367.15076, -353.56052, -419.23618, -424.16266,
       -455.1438 , -439.0756 , -352.38678, -424.1789 , -402.1505 ,
       -457.71317, -387.03192, -385.9121 , -352.38895, -401.91226,
       -461.29

In [ ]:
connect_dist_grad(features)

In [ ]:
pred = network(features)

In [ ]:
print(pred)
type(pred)

tf.Tensor([-3.5069408  2.064058  -5.1743116 ...  0.8846982  0.8917517  1.0203738], shape=(1709,), dtype=float32)


tensorflow.python.framework.ops.EagerTensor

In [ ]:
ind = features['ind_1']
nbatch = tf.reduce_max(ind)+1
print(nbatch)
print(nbatch.numpy())

tf.Tensor(100, shape=(), dtype=int32)
100


In [ ]:
pred = tf.math.unsorted_segment_sum(pred, ind[:,0], nbatch)
pred

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([-38.576496  ,  -3.875589  , -42.721645  , -32.81661   ,
       -11.211855  , -32.937847  ,   5.802003  ,   2.9791598 ,
       -14.393218  ,  -1.3694173 ,  -7.413828  ,  15.58048   ,
       -19.209274  , -17.813517  , -22.206045  , -24.269394  ,
        -5.115548  , -33.52389   , -34.81928   , -18.159735  ,
       -17.229607  ,   4.976848  ,  30.94905   , -25.898273  ,
        -9.5289135 , -16.322235  , -12.8861065 , -20.928131  ,
         0.71945095,   5.581403  , -15.924778  ,   0.21745384,
         6.833212  , -46.58889   , -19.072384  , -28.815624  ,
        26.203857  , -26.21617   ,  26.075907  ,  16.083248  ,
         5.6774693 ,   9.409935  , -14.423883  , -26.329338  ,
       -22.64735   , -44.81467   , -23.316532  , -18.641758  ,
        12.503282  ,   8.121172  , -29.628021  ,  -6.614587  ,
        20.191221  ,   8.790024  , -58.588703  ,  40.155075  ,
        -4.7898626 , -51.048965  ,  31.723305  ,  11.595457  ,
       

In [ ]:
network.summary()

Model: "pi_net_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
preprocess_layer_2 (Preproce multiple                  0         
_________________________________________________________________
polynomial_basis_2 (Polynomi multiple                  0         
_________________________________________________________________
res_update_8 (ResUpdate)     multiple                  80        
_________________________________________________________________
res_update_9 (ResUpdate)     multiple                  0         
_________________________________________________________________
res_update_10 (ResUpdate)    multiple                  0         
_________________________________________________________________
res_update_11 (ResUpdate)    multiple                  0         
_________________________________________________________________
gc_block_8 (GCBlock)         multiple                  177

In [ ]:
from pinn.optimizers import get
optimizer = get(params['optimizer'])
#print(type(optimizer))
#print(type(tf.keras.optimizers.Adam())) #the default_adam optimizer is same class as the tf builtin

mse_loss_fn = tf.keras.losses.MeanSquaredError()
network.compile(optimizer, mse_loss_fn) 



In [ ]:
network.fit()

ValueError: Failed to find data adapter that can handle input: <class 'NoneType'>, <class 'NoneType'>

In [ ]:
network.fit(train(), steps_per_epoch = 100, epochs=3)

Epoch 1/3


2022-04-13 16:18:40.876174: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


InternalError: 2 root error(s) found.
  (0) Internal:  Invalid input shapes
	 [[node mean_squared_error/SquaredDifference (defined at Users/miguelnavaharris/PiNN/pinn/networks/pinet.py:197) ]]
	 [[gradient_tape/pi_net/gc_block_3/ip_layer_3/ones_like/Shape/_226]]
  (1) Internal:  Invalid input shapes
	 [[node mean_squared_error/SquaredDifference (defined at Users/miguelnavaharris/PiNN/pinn/networks/pinet.py:197) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_12028]

Errors may have originated from an input operation.
Input Source operations connected to node mean_squared_error/SquaredDifference:
 pi_net/ann_output/Squeeze (defined at Users/miguelnavaharris/PiNN/pinn/layers.py:244)	
 IteratorGetNext (defined at var/folders/2r/53clw21s5zv7ml4w3_xhd8v40000gq/T/ipykernel_59962/2669695548.py:1)

Input Source operations connected to node mean_squared_error/SquaredDifference:
 pi_net/ann_output/Squeeze (defined at Users/miguelnavaharris/PiNN/pinn/layers.py

In [ ]:
for i in train():
    print(i)
    break

{'elems': <tf.Tensor: shape=(1830,), dtype=int32, numpy=array([8, 6, 6, ..., 1, 1, 1], dtype=int32)>, 'coord': <tf.Tensor: shape=(1830, 3), dtype=float32, numpy=
array([[ 0.18664832,  1.3683072 ,  0.05611368],
       [ 0.0257076 , -0.03380111,  0.12020747],
       [-1.4044749 , -0.48390144,  0.12326425],
       ...,
       [-2.4148157 , -2.523153  ,  2.2402215 ],
       [ 1.2604543 , -0.77396494,  1.8882492 ],
       [ 0.82000977, -1.5932512 , -0.8911141 ]], dtype=float32)>, 'e_data': <tf.Tensor: shape=(100,), dtype=float32, numpy=
array([-458.93304, -365.03094, -437.83813, -439.1224 , -423.01086,
       -387.10318, -403.14987, -436.64743, -424.17764, -424.23813,
       -564.3866 , -439.03198, -403.09518, -352.34125, -347.82965,
       -458.8485 , -453.99692, -437.81097, -382.04916, -358.90292,
       -385.8513 , -416.819  , -422.97424, -348.9662 , -417.97107,
       -421.86795, -475.01062, -348.97452, -424.2242 , -437.88068,
       -439.13327, -388.26315, -368.37668, -419.13052, -398.

In [ ]:
#Instantiate an optimizer
from pinn.optimizers import get
optimizer = get(params['optimizer'])

#Instantiate a loss function
mse_loss_fn = tf.keras.losses.MeanSquaredError()
network.trainable_weights

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = network(x_batch_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = mse_loss_fn(y_batch_train, logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, network.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, network.trainable_weights))

        # Log every 200 batches.
        if step % 100 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * batch_size))

[<tf.Variable 'pi_net_2/res_update_8/dense_77/kernel:0' shape=(5, 16) dtype=float32, numpy=
array([[ 2.03297138e-01,  4.29028571e-01, -1.31578565e-01,
         1.75602496e-01, -3.82390082e-01, -1.17362499e-01,
         5.40828705e-03, -2.29561538e-01, -1.38524562e-01,
         3.99804533e-01, -3.84049892e-01,  3.68797600e-01,
        -6.55816793e-02,  5.21279573e-02,  2.08424330e-02,
        -4.96414304e-01],
       [ 3.74865413e-01, -5.02110898e-01, -3.09326530e-01,
         1.14981413e-01, -1.12918675e-01,  3.08140934e-01,
        -1.89438045e-01,  4.68237698e-01, -1.00623637e-01,
        -1.87570393e-01,  5.05212605e-01,  1.37865543e-04,
         4.36268985e-01, -4.15865064e-01,  1.86822712e-01,
        -4.71323907e-01],
       [-2.71310508e-01, -5.89677989e-02, -1.51873916e-01,
        -3.80368888e-01, -2.32148945e-01, -4.00364250e-01,
        -2.85911322e-01, -3.51970494e-02, -2.69849539e-01,
         2.66469598e-01,  3.74355316e-02, -1.20290071e-01,
         3.41016412e-01, -2.89

Work out loss parameters (see get_train_op) & training schedule (epochs etc), write a training script for Keras using model.fit and then we good